In [21]:
import pandas as pd
import geopandas as gpd

In [51]:
import requests
from bs4 import BeautifulSoup

# Sample posts 

I will first look at one sample post to see where the description is located. 

Sample East Hollywood Post URL: https://losangeles.craigslist.org/lac/apa/d/los-angeles-east-hollywood-open-house/7312988435.html 

In [52]:
sample1 = 'https://losangeles.craigslist.org/lac/apa/d/los-angeles-east-hollywood-open-house/7312988435.html' 
r_sample1 = requests.get(sample1)

In [53]:
sample1_soup = BeautifulSoup(r_sample1.content)
print(sample1_soup.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="craigslist" property="og:site_name"/>
  <meta content="preview" name="twitter:card"/>
  <meta content="EAST HOLLYWOOD) OPEN HOUSE! RENOVATED SINGLE/STUDIO! SMALL PET OK -..." property="og:title"/>
  <meta content="OPEN HOUSE! This unit is located at 542 N. Alexandria Ave. Los Angeles, CA 90004. Renting for $1,350 Per Month with 1,350 Deposit. Any questions please Call or Text Max at or Rosie at It's a..." name="description"/>
  <meta content="OPEN HOUSE! This unit is located at 542 N. Alexandria Ave. Los Angeles, CA 90004. Renting for $1,350 Per Month with 1,350 Deposit. Any questions please Call or Text Max at or Rosie at It's a..." property="og:description"/>
  <meta content="https://images.craigslist.org/00Y0Y_5lIZCIlSDMmz_0CI0t2_600x450.jpg" property="og:image"/>
  <met

Looks like the description is in <section id = "postingbody"> 

# getPosts function

We define a function to take 2 parameters: 1. neighborhood name (string) and 2. number of posts, and return a dataframe of the posts with columns for post title, description, price, neighborhood, bedrooms, SF, post URL, and lat/long coordinates. 

In [61]:
def getPosts(neighborhood, howmany):
    postList = []
    n = neighborhood.replace(' ', '%20')
    
    # range 0 (1st page) - number of posts you want, going up in 120 increments 
    ## looping over the pages until you reach the target HOWMANY posts 
    for i in range(0, howmany, 120): 
    #FOR EACH PAGE: 
        url = f'https://losangeles.craigslist.org/d/housing/search/hhh?s={i}&query=' + n      
        #request the text from the url 
        r =requests.get(url)
        soup = BeautifulSoup(r.content)
        
        #get a list of posts 
        posts = soup.find_all('li', class_ = 'result-row')
        
        #FOR EACH POST 
        for post in posts:
            #store temporary variables 
            price = post.find('span', class_ = 'result-price')
            if price is not None: 
                price = price.text
            
            #break down housingsize into BR and SF 
            housingsize = post.find('span', class_ ='housing')
            bedrooms = None 
            sqft = None
            if housingsize is not None:
                housingsize = housingsize.text
                hlist = housingsize.split()
                
                for h in hlist:
                    if 'br' in h:
                        bedrooms = int(h.replace('br', ''))
                    if 'ft2' in h:
                        sqft = int(h.replace('ft2', ''))
                
            
            neighborhood = post.find('span', class_='result-hood')
            if neighborhood is not None: 
                neighborhood = neighborhood.text
                
            title = post.find('a', class_='result-title')
            if title is not None:
                title = title.text
            
            #go into the individual post's url and get the description...
            url_post = post.find('a', class_='result-title')['href']
            r_post = requests.get(url_post)
            soup_post = BeautifulSoup(r_post.content)
            description = soup_post.find('section', id="postingbody")
            
#             if "no parking" in soup_post.text.lower():
#                 noparking = True
#             elif "parking" in soup_object.text.lower():
#                 noparking = False
#             else:
#                 noparking = True
            
            latlon = soup_post.find('div', class_='viewposting')
            if latlon is not None:
                latitude = latlon['data-latitude']
                longitude = latlon['data-longitude']
            
            postList.append({'Post Title':title, 'Description':description, 'Neighborhood':neighborhood,'Price': price, 'No. Beds':bedrooms, 'SF':sqft, 'URL':url_post, 'lat': latitude, 'lon': longitude})
        
        df = pd.DataFrame(postList)
        
    return df

In [62]:
df_eastholly = getPosts('East Hollywood', 1000)

In [63]:
df_eastholly

,Post Title,Description,Neighborhood,Price,No. Beds,SF,URL,lat,lon
0,"East hollywood 1 bedroom, w'patio,quiet neighb...","[\n, [\n, [QR Code Link to This Post], \n, [],...",(4069 oakwood ave),"$1,595",1.0,NaN,https://losangeles.craigslist.org/lac/apa/d/lo...,34.077810,-118.295191
1,"East hollywood 1 bedroom, w'patio,quiet neighb...","[\n, [\n, [QR Code Link to This Post], \n, [],...",(4069 oakwood ave),"$1,595",1.0,NaN,https://losangeles.craigslist.org/lac/apa/d/lo...,34.077810,-118.295191
2,EAST HOLLYWOOD) OPEN HOUSE! RENOVATED SINGLE/S...,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(542 N. Alexandria Ave),"$1,350",NaN,350.0,https://losangeles.craigslist.org/lac/apa/d/lo...,34.080053,-118.298108
3,EAST HOLLYWOOD AREA) COMING UP! BEAUTIFUL SING...,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(538 N. Alexandria Ave),"$1,350",NaN,350.0,https://losangeles.craigslist.org/lac/apa/d/lo...,34.079963,-118.298108
4,Large 1 bedroom in Beautiful East Hollywood,"[\n, [\n, [QR Code Link to This Post], \n, [],...","(Hollywood, Los Angeles, East Hollywood)","$1,495",1.0,NaN,https://losangeles.craigslist.org/lac/apa/d/lo...,34.089609,-118.310656
...,...,...,...,...,...,...,...,...,...
731,COMMERCIAL WAREHOUSE/OFFICE SPACE FOR LEASE,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(Los Angeles Area),$0,NaN,NaN,https://losangeles.craigslist.org/lac/off/d/lo...,33.863500,-118.180100
732,COMMERCIAL WAREHOUSE/OFFICE SPACE FOR LEASE,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(Los Angeles Area),$0,NaN,NaN,https://losangeles.craigslist.org/lac/off/d/lo...,33.990900,-118.153200
733,COMMERCIAL WAREHOUSE/OFFICE SPACE FOR LEASE,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(Santa Fe Springs Area),$0,NaN,NaN,https://losangeles.craigslist.org/lac/off/d/sa...,33.946400,-118.083800
734,Retail Auto Dealer Lot Office & Used Car Spot|...,"[\n, [\n, [QR Code Link to This Post], \n, [],...",None,$0,NaN,NaN,https://losangeles.craigslist.org/lac/off/d/lo...,34.049300,-118.291700


In [65]:
df_500 = getPosts('East Hollywood', 500)
df_500

,Post Title,Description,Neighborhood,Price,No. Beds,SF,URL,lat,lon
0,"East hollywood 1 bedroom, w'patio,quiet neighb...","[\n, [\n, [QR Code Link to This Post], \n, [],...",(4069 oakwood ave),"$1,595",1.0,NaN,https://losangeles.craigslist.org/lac/apa/d/lo...,34.077810,-118.295191
1,"East hollywood 1 bedroom, w'patio,quiet neighb...","[\n, [\n, [QR Code Link to This Post], \n, [],...",(4069 oakwood ave),"$1,595",1.0,NaN,https://losangeles.craigslist.org/lac/apa/d/lo...,34.077810,-118.295191
2,EAST HOLLYWOOD) OPEN HOUSE! RENOVATED SINGLE/S...,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(542 N. Alexandria Ave),"$1,350",NaN,350.0,https://losangeles.craigslist.org/lac/apa/d/lo...,34.080053,-118.298108
3,EAST HOLLYWOOD AREA) COMING UP! BEAUTIFUL SING...,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(538 N. Alexandria Ave),"$1,350",NaN,350.0,https://losangeles.craigslist.org/lac/apa/d/lo...,34.079963,-118.298108
4,Large 1 bedroom in Beautiful East Hollywood,"[\n, [\n, [QR Code Link to This Post], \n, [],...","(Hollywood, Los Angeles, East Hollywood)","$1,495",1.0,NaN,https://losangeles.craigslist.org/lac/apa/d/lo...,34.089609,-118.310656
...,...,...,...,...,...,...,...,...,...
595,******548 S Main St! TOPAZ! ~H~U~G~E~ GRND FLR...,"[\n, [\n, [QR Code Link to This Post], \n, [],...","(DTLA, Downtown LA, Los Angeles)","$27,000",NaN,13500.0,https://losangeles.craigslist.org/lac/off/d/lo...,34.045832,-118.249068
596,***501 S LA St! DTLA's Toy District GRND FLR O...,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(Los Angeles),"$2,596",NaN,1300.0,https://losangeles.craigslist.org/lac/off/d/lo...,34.045895,-118.247942
597,453 Spring #1029- 24/7 Access Creative Office ...,"[\n, [\n, [QR Code Link to This Post], \n, [],...",(Los Angeles),"$1,180",NaN,590.0,https://losangeles.craigslist.org/lac/off/d/lo...,34.048116,-118.248950
598,TOPAZ! Units A! LIKE-NEW! GRND FLR Commercial ...,"[\n, [\n, [QR Code Link to This Post], \n, [],...","(DTLA, Downtown LA, Los Angeles)","$2,021",NaN,1155.0,https://losangeles.craigslist.org/lac/off/d/lo...,34.045352,-118.248382


okay why is this giving me 600 rows????
Try it with a multiple of 120 I guess? 

In [71]:
df_eastholly.loc[0]['URL']

'https://losangeles.craigslist.org/lac/apa/d/los-angeles-east-hollywood-bedroom/7313009573.html'

In [70]:
df_eastholly.loc[0]['Description']

<section id="postingbody">
<div class="print-information print-qrcode-container">
<p class="print-qrcode-label">QR Code Link to This Post</p>
<div class="print-qrcode" data-location="https://losangeles.craigslist.org/lac/apa/d/los-angeles-east-hollywood-bedroom/7313009573.html"></div>
</div>
4069 Oakwood Ave.<br/>
Los Angeles, CA 90004<br/>
<br/>
Text / Call Frank @  <a class="show-contact" data-href="/reply/lax/apa/7313009573/__SERVICE_ID__" href="#" rel="nofollow" title="click to show contact info">show contact info</a>
<br/>
<br/>
FALL PROMO 100 dollars off for 12 months, Reduced rent is 1595 rather than 1695<br/>
<br/>
Rent: $1,595.00<br/>
<br/>
<br/>
<br/>
This is a beautifully remodeled 1 Bedroom, 1 bath apartment, with new designer hardwood Floors, black granite countertops and a balcony.<br/>
<br/>
The building is centrally located in a quiet, charming neighborhood in East Hollywood.<br/>
<br/>
Walking distance to Ralphs, Vons, Starbucks, and tons of restaurants! Also a 5 minut

Yayy the descriptions are successfully pulled! 